#### Импорт необходимых библиотек

In [ ]:
import json
import os
import jiwer as jiwer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from vosk import Model, KaldiRecognizer
import wave
import numpy as np
from pydub import AudioSegment
from sklearn.feature_extraction.text import TfidfVectorizer


# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"  # Путь к скачанной модели Vosk

#### Объявление классов

In [ ]:
# Датасет для классификации текста
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [ ]:
# Нейронная сеть для классификации текста
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

##### Объявление функций

In [ ]:
# Функция для обработки аудиофайла
def transcribe_audio(audio_file, dir):
    wf = wave.open(f"{dir}/{audio_file}", "rb")
    rec = KaldiRecognizer(model, wf.getframerate())

    result_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            result_text += result.get("text", "")

    final_result = json.loads(rec.FinalResult())
    result_text += final_result.get("text", "")

    return result_text

In [ ]:
# Функция классификации текста
def classify_text(text, classifier, tokenizer):
    # Преобразование текста в вектор
    text_vector = tokenizer.transform([text]).toarray()
    text_tensor = torch.tensor(text_vector, dtype=torch.float32)

    # Классификация текста
    with torch.no_grad():
        outputs = classifier(text_tensor)
        
    _, predicted_class = torch.max(outputs, 1)

    return predicted_class.item()

In [ ]:
# Функция для обработки аудиофайлов
def process_audio_files(file_list, classifier, tokenizer, dir):
    results = []
    for file_info in file_list:
        # [{'file': file_name, 'id': file_id}, ...]
        print(f"pocessing {file_info['file']}")
        audio_file = file_info['file']
        audio_id = file_info['id']

        # Распознавание текста из аудио
        transcribed_text = transcribe_audio(audio_file, dir)

        # Классификация текста
        predicted_class = classify_text(transcribed_text, classifier, tokenizer)

        # Извлечение атрибута
        attribute = any(char.isdigit() for char in transcribed_text)

        result = {
            "audio_filepath": os.path.basename(audio_file),
            "id": audio_id,
            "text": transcribed_text,
            "category": predicted_class,
            "attribute": attribute
        }

        results.append(result)
        print(f"done {file_info['file']}")

    return results

##### Подготовка настроек модели, создание модели

In [ ]:
# Пример классификатора и токенизатора
input_dim = 2  # Зависит от метода векторизации текста
hidden_dim = 55
output_dim = 22  # Количество классов
classifier = TextClassifier(input_dim, hidden_dim, output_dim)

In [ ]:
model = Model(MODEL_PATH)

#### Подготовка данных для обучения

In [ ]:
train_data_dir = '../data/train/annotation'
with open(f'{train_data_dir}/hr_bot_synt.json', 'r', encoding='utf-8') as f:
    training_annotations = json.load(f)


def load_data_annotations():
    audio_files = []
    texts = []  # Замените на свои данные
    labels = []  # Метки классов


    for annotation in training_annotations:
        audio_filepath = annotation['audio_filepath']
        file_id = annotation['id']
        text = annotation['text']
        label = annotation['label']

        # Добавление данных в соответствующие списки
        audio_files.append({"file": audio_filepath, "id": file_id})
        texts.append(text)
        labels.append(label)

    return audio_files, texts, labels

audio_files, texts, labels = load_data_annotations()

# Разделение данных на тренировочные и тестовые
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

tokenizer = TfidfVectorizer(max_features=input_dim)

# Подготовка текстовых данных для классификации
train_vectors = tokenizer.fit_transform(train_texts).toarray()
test_vectors = tokenizer.transform(test_texts).toarray()

train_dataset = TextDataset(train_vectors, train_labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

#### Функция обучения модели (+ расчет метрик каждые 5 эпох)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

val_reference_texts = []
val_audio_files = []

def train():
    for epoch in range(30):
        for texts_batch, labels_batch in train_loader:
            texts_batch = texts_batch.float()
            optimizer.zero_grad()
            outputs = classifier(texts_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

        # Оценка на валидационном наборе после каждой эпохи
        if (epoch + 1) % 5 == 0:  # Оценка каждые 5 эпох
            # val_hypothesis_texts = process_audio_files(val_audio_files, classifier, tokenizer, dir)

            # # Рассчитываем WER для валидационного набора
            # wer = jiwer.wer(val_reference_texts, val_hypothesis_texts)
            print(f"Эпоха {epoch + 1}")

##### Использование модели для транскрибации и классификации текста

In [ ]:
# директория, где лежат файлы, которые мы хотим обработать
result_dir = '../data/train/hr_bot_synt'

result_files = os.listdir(result_dir)

for i in range(len(result_files)):
    result_files[i] = {"file": result_files[i], "id": result_files[i][:-4]}

# Обработка файлов и получение результатов
transcription_results = process_audio_files(result_files, classifier, tokenizer, result_dir)

# Сохранение результатов в JSON
with open("transcriptions.json", "w", encoding="utf-8") as f:
    json.dump(transcription_results, f, ensure_ascii=False, indent=4)

Вывод результатов в консоль

In [ ]:
# выводим результат
# print(json.dumps(transcription_results, indent=4, ensure_ascii=False))